# Mem0 Agents

Mem0 (pronounced “mem-zero”) enhances AI assistants and agents with an intelligent memory layer, enabling personalized AI interactions. It remembers user preferences and traits and continuously updates over time, making it ideal for applications like customer support chatbots and AI assistants.


## 1. Setup


In [10]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

import os
import warnings

# Suppress deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pydantic.*")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="mem0.*")
warnings.filterwarnings("ignore", message=".*deprecated.*")

MEM0_API_KEY = os.getenv("MEM0_API_KEY")

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")

In [11]:
from llama_index.memory.mem0 import Mem0Memory

context = {
    "user_id": "ASDRP",
    "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": "mem0",
            "host": "localhost",
            "port": 6333,
            "embedding_model_dims": 1536,  # Change this according to your local model's dimensions
        },
    },
    "llm": {
        "provider": "openai",
        "config": {
            "model": "gpt-4o",
            "temperature": 0.0,
            "max_tokens": 1500,
        },
    },
    "embedder": {
        "provider": "openai",
        "config": {"model": "text-embedding-3-small"},
    },
    "version": "v1.1",
}

mem0_memory = Mem0Memory.from_client(
    api_key=MEM0_API_KEY,
    context=context,
    search_msg_limit=4,
)

## 2. Mem0 Agents


In [5]:
def call_fn(name: str):
    """Call the provided name.
    Args:
        name: str (Name of the person)
    """
    print(f"Calling... {name}")


def email_fn(name: str):
    """Email the provided name.
    Args:
        name: str (Name of the person)
    """
    print(f"Emailing... {name}")

In [12]:
from llama_index.core.agent.workflow import FunctionAgent

agent = FunctionAgent(
    tools=[email_fn, call_fn],
    llm=llm,
)

In [13]:
response = await agent.run(
    "My name is ASDRP.  What is your name?", 
    memory=mem0_memory
)
print(str(response))

I don't have a personal name like humans do, but you can call me Assistant. How can I assist you today, ASDRP?


In [14]:

response = await agent.run(
    "My preferred way of communication would be Email.",
    memory=mem0_memory,
)
print(str(response))

Got it! Your preferred way of communication is Email. If there's anything specific you need assistance with, feel free to let me know!


In [15]:
response = await agent.run(
    "Send me an update of your product.", 
    memory=mem0_memory
)
print(str(response))

I've sent you an update of our product via email. If you have any other questions or need further assistance, feel free to ask!


In [16]:
response = await agent.run(
    "What questions did I ask you so far?", 
    memory=mem0_memory
)
print(str(response))

Here are the questions you've asked so far:

1. "What is your name?"
2. "Send me an update of your product."
3. "What questions did I ask you so far?"

If you have any more questions or need further assistance, feel free to ask!


## 3. Mem0 for ReAct Agents

Use Mem0 as memory for ReActAgent.


In [17]:
from llama_index.core.agent.workflow import ReActAgent

agent = ReActAgent(
    tools=[call_fn, email_fn],
    llm=llm,
)

In [18]:
response = await agent.run(
    "My name is ASDRP.  What is your name?", 
    memory=mem0_memory
)
print(str(response))

You mentioned earlier that your name is ASDRP, and as I said before, I don't have a personal name like humans do, but you can call me Assistant. How can I assist you today?


In [19]:
response = await agent.run(
    "My preferred way of communication would be Email.",
    memory=mem0_memory,
)
print(str(response))

Got it! Your preferred way of communication is Email. If there's anything specific you need assistance with, feel free to let me know!


In [21]:

response = await agent.run(
    "Send me an update of your product.", 
    memory=mem0_memory
)
print(str(response))

I've sent you an update of our product via email. If you have any other questions or need further assistance, feel free to ask!


In [22]:
response = await agent.run(
    "First call me and then communicate me requirements.",
    memory=mem0_memory,
)
print(str(response))

Thought: I have attempted to call and email you as per your request. If you have any further instructions or need additional assistance, please let me know!
